# Strands AgentでOpenAIモデルを使用する

## 概要

Strands Agentsは、数行のコードでAIエージェントを構築・実行するモデル駆動型SDKです。Strandsは複数のプロバイダーとどこでホストされていても使用可能なモデルをサポートしています。

[LiteLLM](https://docs.litellm.ai/docs/)は、Amazon、Anthropic、OpenAIなど様々なLLMプロバイダーに対して統一されたインターフェースを提供し、単一のAPI経由でこれらのモデルと対話できるようにします。Strands Agent SDKはLiteLLMプロバイダーを実装しており、LiteLLMがサポートするあらゆるモデルに対してエージェントを実行できます。

この例では、Strands Agentの基盤モデルとしてMicrosoft Azureでホストされている`gpt-4.1-mini`モデルを使用する方法を示します。天気と時刻取得ツールを使用したシンプルなエージェントユースケースを使用します。


## エージェント詳細
<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                              |
|--------------------|---------------------------------------------------|
|使用機能            |LiteLLMモデル                                      |
|エージェント構造    |シングルエージェントアーキテクチャ                  |

</div>

## アーキテクチャ

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 主な機能
* **LiteLLMモデル**: LiteLLM経由で提供されるモデルを使用

## セットアップと前提条件

### 前提条件
* Python 3.10+
* Azureアカウント
* gpt-4.1-miniへのアクセス

Strands Agentに必要なパッケージをインストールしましょう

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

### 依存パッケージのインポート

依存パッケージをインポートしましょう

In [ ]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel

### Azureキーの設定

Azure APIキーを設定しましょう

In [ ]:
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

### カスタムツールの設定

エージェントをテストするための2つのダミーツールを作成しましょう

In [ ]:
@tool
def current_time(timezone: str = "UTC") -> str:
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    # Dummy implementation. Please replace with actual weather API call.
    return "sunny"

### エージェントの基盤LLMモデルの定義

次に、LiteLLMを使用してエージェントの基盤モデルを定義します。`gpt-4.1-mini`に設定します

In [ ]:
model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)

### エージェントの定義

必要な情報がすべて揃いました。エージェントを定義しましょう

In [ ]:
system_prompt = "You are a simple agent that can tell the time and the weather"
agent = Agent(
    model=litellm_model,
    system_prompt=system_prompt,
    tools=[current_time, current_weather],
)

### エージェントのテスト

エージェントをテストするために呼び出してみましょう

In [ ]:
results = agent("What time is it in Seattle? And how is the weather?")

#### エージェントの結果分析

素晴らしい！初めてエージェントを呼び出しました！結果オブジェクトを詳しく見てみましょう。最初に、エージェントオブジェクト内のメッセージ交換を確認できます

In [ ]:
agent.messages

次に、最後のクエリに対するエージェントの使用状況を`metrics`で分析できます

In [ ]:
results.metrics

### おめでとうございます！

このノートブックで、LiteLLMを使用してOpenAIモデルで天気エージェントの回答を提供する方法を学びました。